In [51]:
import os
import torchvision
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

DATA_PATH = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages"

from torchvision.datasets.folder import default_loader
class CustomDatasetFolder(datasets.DatasetFolder):
    def __init__(self, root, loader=default_loader, extensions=None, transform=None, target_transform=None, is_valid_file=None):
        super(CustomDatasetFolder, self).__init__(root, loader, extensions, transform, target_transform, is_valid_file)

    def find_ignore_indices(self, ignore_paths):
        ignore_indices = []
        for ignore_path in ignore_paths:
            for i, (path, _) in enumerate(self.samples):
                if path == ignore_path:
                    ignore_indices.append(i)
        return ignore_indices

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        if self.transform is not None:
            sample = self.transform(sample)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    def update_dataset(self, ignore_paths):
        ignore_indices = self.find_ignore_indices(ignore_paths)

        # Collect samples and targets to keep
        updated_samples = [sample for i, sample in enumerate(self.samples) if i not in ignore_indices]
        updated_targets = [target for i, target in enumerate(self.targets) if i not in ignore_indices]

        # Update the dataset
        self.samples = updated_samples
        self.targets = updated_targets

IGNORE_PATHS = [
    '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg',
    '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg']

In [52]:
import torchvision
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define your transformation
transformation_steps = transforms.Compose([
    transforms.Resize((64, 64)),  # Adjust the size as needed
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

# Load your dataset
dataset = CustomDatasetFolder(
    root=DATA_PATH, 
    loader=torchvision.datasets.folder.default_loader, 
    transform=transformation_steps,
    extensions=['jpg']
)
dataset.update_dataset(IGNORE_PATHS)

# Define the split ratios
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Calculate the number of samples for each split
num_train = int(train_ratio * len(dataset))
num_test = len(dataset) - num_train

# Split the dataset
train_set, test_set = random_split(dataset, [num_train, num_test])

# Create data loaders
BATCH_SIZE = 1
NUM_WORKERS = os.cpu_count()
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [53]:
from torch import nn
import torch.nn.functional as F

class DogCatClassifier(nn.Module):
    def __init__(self, 
               input_shape: int,
               output_shape: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        # Global Average Pooling layer
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(1)

        # Fully connected layers
        self.fc1 = nn.Linear(256, 1024)
        self.fc2 = nn.Linear(1024, 2)  # Assuming 2 classes for the final layer


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        x = self.global_avg_pooling(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor

        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)

        return x

In [54]:
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import StepLR
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = DogCatClassifier(
    input_shape=3,
    output_shape=2).to(device)

loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.0001
)
# step_size = 5
# gamma = 0.1
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
# Set number of epochs
EPOCHS = 20
total_dataset_size = len(train_dataloader)

for epoch in tqdm(range(EPOCHS)):
    model.train()
    # Training
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(train_dataloader):
        if (batch // BATCH_SIZE) % 100 == 0:
            print(f"\rEpoch: {epoch}/{EPOCHS} | Batch: {batch}/{total_dataset_size}", end="", flush=True)
        X, y = X.to(device), y.to(device)
        y_pred = model(X) # 1. Forward pass
        loss = loss_fn(y_pred, y) # 2. Calculate the loss
        train_loss += loss.item()
        optimizer.zero_grad() # 3. Optimizer zero grad
        loss.backward() # 4. Loss backward
        optimizer.step()# 5. Optimizer step
        # Calculate accuracy metric
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class==y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(train_dataloader)
    train_acc = train_acc / len(train_dataloader) 
    
    # Testing
    test_loss, test_acc = 0,0
    model.eval()
    with torch.inference_mode():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        test_loss = test_loss / len(test_dataloader)
        test_acc = test_acc / len(test_dataloader)
    print(f" | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0/20 | Batch: 19900/19998 | Train loss: 0.6799 | Train acc: 0.5569 | Test loss: 0.6511 | Test acc: 0.6190
Epoch: 1/20 | Batch: 19900/19998 | Train loss: 0.6540 | Train acc: 0.6159 | Test loss: 0.6345 | Test acc: 0.6496
Epoch: 2/20 | Batch: 19900/19998 | Train loss: 0.6401 | Train acc: 0.6385 | Test loss: 0.6181 | Test acc: 0.6674
Epoch: 3/20 | Batch: 19900/19998 | Train loss: 0.6488 | Train acc: 0.6351 | Test loss: 0.6289 | Test acc: 0.6422
Epoch: 4/20 | Batch: 19900/19998 | Train loss: 0.6205 | Train acc: 0.6622 | Test loss: 0.6325 | Test acc: 0.6362
Epoch: 5/20 | Batch: 19900/19998 | Train loss: 0.6144 | Train acc: 0.6687 | Test loss: 0.6101 | Test acc: 0.6754
Epoch: 6/20 | Batch: 19900/19998 | Train loss: 0.6086 | Train acc: 0.6778 | Test loss: 0.5989 | Test acc: 0.6952
Epoch: 7/20 | Batch: 19900/19998 | Train loss: 0.6058 | Train acc: 0.6817 | Test loss: 0.5951 | Test acc: 0.6940
Epoch: 8/20 | Batch: 19900/19998 | Train loss: 0.5958 | Train acc: 0.6937 | Test loss: 0.6298 | 

In [55]:
# working 
# lr = 0.0001, epochs = 20, ADAM



# Testing
# changed the batch size form 1 to 64 and resize from 64x64 to 150x150